In [ ]:
import numpy as np
import pandas as pd
import scipy.stats   
import scipy.special  
import subprocess
import sklearn.linear_model
import sklearn.model_selection
import sklearn.pipeline  
import sklearn.preprocessing
import sklearn.ensemble  
import sklearn.kernel_ridge 
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
color = sns.color_palette()
sns.set_style('darkgrid')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

In [ ]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/train.csv')
df = pd.concat([train_df, test_df])
print('train shape: ', train_df.shape)
print('Test shape:', test_df.shape)
print('df shape:', df.shape)
df

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.head(3)

In [ ]:
print(train_df['Survived'].describe())
sns.distplot(train_df['Survived'], fit=scipy.stats.norm)

In [ ]:
print(df.isnull().sum())
cols_with_missing = [col for col in df.columns 
                                 if df[col].isnull().any()]
print("Columns with missing features: ",  cols_with_missing)

# Treat null values

In [ ]:
imputer = sklearn.preprocessing.Imputer()

In [ ]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()

# Drop columns where large number of values are null
columns_to_drop = ['Age', 'Cabin', 'Name', 'Ticket']
df = df.drop(columns_to_drop, axis=1)

df[['Embarked']] = df[['Embarked']].fillna(value='None')
df[['Fare']] = df[['Fare']].fillna(value=0)
df[['Survived']] = df[['Survived']].fillna(value=0.0)

In [ ]:
# categorial features
cols = ('Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp')
# shape        
print('before data_df.shape = ', df.shape)
# apply sklearn.preprocessing.LabelEncoder to each categorical feature
for c in cols:
    lbl = sklearn.preprocessing.LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))
    
# shape
print('after data_df.shape = ', df.shape)
df

In [ ]:
cols = df.select_dtypes(exclude =[np.number]).columns.values
df  = pd.get_dummies(df).copy()

In [ ]:
train_df = df[: train_df.shape[0]]
train_df_y = train_df[['Survived']]
train_df = train_df.drop('Survived', axis=1)

In [250]:
test_df = df[train_df.shape[0]-1:]
test_df = test_df.drop('Survived', axis=1)

In [251]:
test_df.shape

(892, 7)

In [252]:
test_df.loc[0:]
test_df

PassengerId  Pclass  Sex  SibSp  Parch  Fare  Embarked
890          891       2    1      0      0    30         2
0              1       2    1      1      0    18         3
1              2       0    0      1      0   207         0
2              3       2    0      0      0    41         3
3              4       0    0      1      0   189         3
4              5       2    1      0      0    43         3
5              6       2    1      0      0    51         2
6              7       0    1      0      0   186         3
7              8       2    1      3      1   124         3
8              9       2    0      0      2    74         3
9             10       1    0      1      0   154         0
10            11       2    0      1      1   110         3
11            12       0    0      0      0   143         3
12            13       2    1      0      0    43         3
13            14       2    1      1      5   158         3
14            15       2    0      0      0    36         3
15            16       1    0      0      0   108         3
16            17       2    1      4      1   151         2
17            18       1    1      0      0    85         3
18            19       2    0      1      0   113         3
19            20       2    0      0      0    16         0
20            21       1    1      0      0   138         3
21            22       1    1      0      0    85         3
22            23       2    0      0      0    42         2
23            24       0    1      0      0   168         3
24            25       2    0      3      1   124         3
25            26       2    0      1      5   159         3
26            27       2    1      0      0    16         0
27            28       0    1      3      2   246         3
28            29       2    0      0      0    38         2
29            30       2    1      0      0    40         3
30            31       0    1      0      0   145         0
31            32       0    0      1      0   236         0
32            33       2    0      0      0    30         2
33            34       1    1      0      0    72         3
34            35       0    1      1      0   220         0
35            36       0    1      1      0   187         3
36            37       2    1      0      0    17         0
37            38       2    1      0      0    43         3
38            39       2    0      2      0   113         3
39            40       2    0      1      0    75         0
40            41       2    0      1      0    62         3
41            42       1    0      1      0   123         3
42            43       2    1      0      0    40         0
43            44       1    0      1      2   177         0
44            45       2    0      0      0    38         2
45            46       2    1      0      0    43         3
46            47       2    1      1      0   102         2
47            48       2    0      0      0    30         2
48            49       2    1      2      0   125         0
..           ...     ...  ...    ...    ...   ...       ...
841          842       1    1      0      0    72         3
842          843       0    0      0      0   157         0
843          844       2    1      0      0     4         0
844          845       2    1      0      0    54         3
845          846       2    1      0      0    22         3
846          847       2    1      6      2   205         3
847          848       2    1      0      0    40         0
848          849       1    1      0      1   162         3
849          850       0    0      1      0   224         0
850          851       2    1      4      2   158         3
851          852       2    1      0      0    31         3
852          853       2    0      1      1   101         0
853          854       0    0      0      1   173         3
854          855       1    0      1      0   138         3
855          856       2    0      0      1  

In [253]:
model = sklearn.ensemble.GradientBoostingRegressor(
    n_estimators=10000, 
    learning_rate=0.01,                
    max_depth=5,
    max_features='sqrt',
    min_samples_leaf=15, 
    min_samples_split=10,
    loss='huber', 
    random_state=5
)

In [ ]:
model.fit(train_df, train_df_y.values.ravel())

In [ ]:
predictions = model.predict(test_df)

In [ ]:
submission = pd.DataFrame()
indexes = [i for i in range(892, 1309)]
print(indexes)

len(indexes)
#submission['PassengerId'] = indexes
#submission['Survived'] = predictions
#submission.to_csv('submission3.csv', index=True)
#submission.head(5)